In [ ]:
df_all = df_all.append(df_new,sort=True)
df_all

In [ ]:
df_new = pd.DataFrame(index=['datetime'],
                    columns=['id', 
                            'side', 
                            'price', 
                            'size', 
                            'exec_date', 
                            'buy_child_order_acceptance_id', 
                            'sell_child_order_acceptance_id'])
df_all = df_new.dropna()

In [ ]:
# pip install websocket-client
import websocket
import json
import pandas as pd
import threading
import numpy as np
import time
from datetime import datetime as dt
from datetime import timedelta
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob

unknown_files = len(sorted(glob.glob('./data/plots/unknown/*.png')))
up_files = len(sorted(glob.glob('./data/plots/up/*.png')))
down_files = len(sorted(glob.glob('./data/plots/down/*.png')))


CHANNEL = "lightning_executions_FX_BTC_JPY"
#描画秒数
plot_size=30
#予測先時間
pred_distance=10

i = unknown_files+up_files+down_files+1
threshold_plus= 100
threshold_minus = -100



df_new = pd.DataFrame(index=['datetime'],
                    columns=['id', 
                            'side', 
                            'price', 
                            'size', 
                            'exec_date', 
                            'buy_child_order_acceptance_id', 
                            'sell_child_order_acceptance_id'])
df_all = df_new.dropna()
df_T=pd.DataFrame(index=['Time_index'])


def on_message(ws, message):
    global df_new, i, df_T, df_all, pred_price, tail_price, df_heat_plot_resize
   #取得情報をjsonで代入
    message = json.loads(message)
    #jsonのmethodがchannnelMessageの時
    if message["method"] == "channelMessage":
        print("1")
        #params_messageのparamsとmessageをparams_message（取得したい情報）へ代入
        params_message = message["params"]["message"]
        print("1.1")
        #df_newに取得したい情報のdfを作成する
        df_new = pd.DataFrame(params_message)
        print("1.2")
        #df_newのexec_dateに現在タイムゾーン変換した時間を代入する
        df_new['exec_date'] = pd.to_datetime(df_new['exec_date']) + timedelta(hours=9)
        print("1.3")
        try:
            df_all
#            print(df_all)
        except NameError:
            df_all = df_new#.dropna()
            print("44")
        #df_allにdf_newを追加する
        df_all = df_all.append(df_new,sort=True)
        print("1.4")
        #df_allにdf_newをインデックスに指定する
        df_all.index = df_all['exec_date']
        print("1.5")
        #先頭時間を取得する
        head_time=df_all['exec_date'].iloc[1]
        print("1.6")
        #末尾時間を取得する
        tail_time=df_all['exec_date'].iloc[-1]
        print("1.7")


        window = pd.Timestamp(head_time + timedelta(seconds=1*(plot_size+pred_distance)) + timedelta(hours=0))

        print("2")
        print("head")
        print(head_time)
        print("tail")
        print(tail_time)
        print("window")
        print(window)
        if tail_time - head_time > timedelta(seconds=1.5*plot_size):
            del df_new, df_T, head_time, tail_time, df_all, window, df_heat_plot_resize,df_heat_plot

            df_new = pd.DataFrame(index=['datetime'],
                    columns=['id', 
                            'side', 
                            'price', 
                            'size', 
                            'exec_date', 
                            'buy_child_order_acceptance_id', 
                            'sell_child_order_acceptance_id'])
            df_all = df_new.dropna()
            df_T=pd.DataFrame(index=['Time_index'])
            print("33")




         #観測窓の設定
        if tail_time > window:
            print("3")

#             #取得したデータの先頭からplot_size分をheatmapの形にする
            
# #---------------------------------------------------------------------------------------------------
            df_plot = df_all
            df_plt=df_all.rename(columns={'exec_date':'Index_Time'})
            df_plt['Index_Time']= pd.to_datetime(df_plt['Index_Time'].dt.round("1s"))

            df_plt['size']= df_plt['size']/0.001
            df_plt['size'] = np.log10(df_plt['size']).astype(float)


            df_plt['side'] = df_plt['side'].replace({'BUY':'-1', 'SELL':'1'})
            df_plt['size'] = df_plt['side'].astype(int)*df_all['size']

            df_heat_plot = df_plt.pivot_table(values = ['size'],
                index = ['Index_Time'], columns = ['price']).fillna(0)
            
            df_heat_plot_resize = df_heat_plot.iloc[:-10,:]
            print("4")

    
    
            print("6")

            df_pivot_price = df_plt.pivot_table(values = ['price'],
                index = ['Index_Time'],
                aggfunc = 'mean').round()
            # print("7")

            tail_price=df_pivot_price.iloc[-1*pred_distance]
            #末尾時間を取得する
            pred_price=df_pivot_price.iloc[-1]
            print("8")
            #print('tail time')
            # print(pred_price)
            #print(tail_price)
            #print('window time')
            # print(pred_price)
            #print(window)

            i = i + 1
            print("9")

# #-----------------------------------------------------------------------------------------------------


            if (pred_price['price'] - tail_price['price']) > threshold_plus:
                sns.heatmap(df_heat_plot_resize, cmap="seismic")
                print("5")
    

                plt.savefig('./data/plots/up/{}.png'.format(i))
                plt.show()

                plt.close('all')
                time.sleep(1)
                flag="up"
                print("22")
            elif (pred_price['price'] - tail_price['price']) < threshold_minus:
                sns.heatmap(df_heat_plot_resize, cmap="seismic")
                print("5")    
                plt.savefig('./data/plots/down/{}.png'.format(i))
                plt.show()


                plt.close('all')
                time.sleep(1)
                flag="down"

            else:
                sns.heatmap(df_heat_plot_resize, cmap="seismic")
                print("5")
                plt.savefig('./data/plots/unknown/{}.png'.format(i))
    
                plt.show()
                plt.close('all')
                time.sleep(1)
                flag="unknown"

            print('this_plt_{}'.format(i))
            print(flag)
     
            df_new = pd.DataFrame(index=['datetime'],
                    columns=['id', 
                            'side', 
                            'price', 
                            'size', 
                            'exec_date', 
                            'buy_child_order_acceptance_id', 
                            'sell_child_order_acceptance_id'])
            df_all = df_new.dropna().iloc[1:2,:]
            df_T=pd.DataFrame(index=['Time_index'])
            print("11")
        

        return df_all, i, pred_price, tail_price, df_heat_plot_resize
        
        print("12")


def on_open(ws):
    ws.send(json.dumps({"method": "subscribe",
                        "params": {"channel": CHANNEL}}))

def main():
    ws = websocket.WebSocketApp("wss://ws.lightstream.bitflyer.com/json-rpc",
                                on_message=on_message, on_open=on_open)
    ws.run_forever()
    
if __name__ == "__main__":
    try:
        main()

    except KeyboardInterrupt:
        sys.exit()


fatal: No configured push destination.
Either specify the URL from the command-line or configure a remote repository using

    git remote add <name> <url>

and then push using the remote name

    git push <name>

